In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score,adjusted_rand_score
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer


In [ ]:
train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")
submission_df = pd.read_csv("sample_submission.csv")

train_df.head()

In [ ]:
train_df.columns

In [ ]:
train_df.describe()

In [ ]:
train_df.corr()

In [ ]:
import seaborn as sns
sns.heatmap(train_df.corr())

In [ ]:
cols = train_df.columns

NUM = (train_df._get_numeric_data().columns).to_list()
CAT= cols.difference(NUM).to_list()

In [ ]:
input_cols=[ 'Gender', 'Age', 'Education_Level',
       'Relationship_Status', 'Hometown', 'Unit', 'Decision_skill_possess',
       'Time_of_service', 'Time_since_promotion', 'growth_rate', 'Travel_Rate',
       'Post_Level', 'Pay_Scale', 'Compensation_and_Benefits',
       'Work_Life_balance', 'VAR1', 'VAR2', 'VAR3', 'VAR4', 'VAR5', 'VAR6',
       'VAR7']

In [ ]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999,inplace=True)


In [ ]:
X_train = train_df[input_cols]
y_train = train_df.Attrition_rate

#y_train_log=np.log1p(y_train)

X_test=test_df[input_cols]

In [ ]:
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]

In [ ]:
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=5, depth=3, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train,cat_features=categorical_features_indices)

## Training Loss
train_preds = model.predict(X_train)
train_loss = r2_score(y_train,train_preds)
print(train_loss)

pred_val = model.predict(X_test)

In [ ]:
def submit(pred_val, test_df, filename = 'submission_1d.csv'):
    ## Employee_ID   Attrition_rate
    empId = test_df['Employee_ID'].tolist()
    dict = {"Employee_ID": empId, "Attrition_rate": pred_val}
    sub = pd.DataFrame(dict)
    sub.to_csv(filename, index=False)
submit(pred_val, test_df)